## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-25-14-32-13 +0000,nyt,"Keeping the House Absent, Johnson Marginalizes...",https://www.nytimes.com/2025/10/25/us/politics...
1,2025-10-25-14-13-39 +0000,nyt,Timothy Mellon Is Donor Who Gave $130 Million ...,https://www.nytimes.com/2025/10/25/us/politics...
2,2025-10-25-14-11-25 +0000,bbc,Hundreds of smartphones intensified India bus ...,https://www.bbc.com/news/articles/c15pnlz3gdvo...
3,2025-10-25-14-07-00 +0000,nyt,‘It’s Got to Be an Inside Job’: Jewelry Thieve...,https://www.nytimes.com/2025/10/25/world/europ...
4,2025-10-25-14-05-22 +0000,nypost,Stephen A. Smith refuses to back down after Ka...,https://nypost.com/2025/10/25/sports/stephen-a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
85,trump,55
177,trade,15
53,will,13
70,new,12
335,he,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2025-10-24-21-43-10 +0000,nypost,Ontario to pause anti-tariff Reagan ads after ...,https://nypost.com/2025/10/24/us-news/ontario-...,135
207,2025-10-24-19-57-00 +0000,wsj,Ontario to Pause Anti-Tariff Ad Campaign After...,https://www.wsj.com/world/americas/ontario-to-...,128
219,2025-10-24-19-27-00 +0000,wsj,Ontario to Pause Anti-Tariff Commercial After ...,https://www.wsj.com/world/americas/ontario-to-...,120
225,2025-10-24-18-51-56 +0000,wapo,Trump cancels trade negotiations with Canada o...,https://www.washingtonpost.com/world/2025/10/2...,109
18,2025-10-25-13-07-30 +0000,bbc,Trump hopes China will help push Russia toward...,https://www.bbc.com/news/articles/cn51yk9vgrko...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,135,2025-10-24-21-43-10 +0000,nypost,Ontario to pause anti-tariff Reagan ads after ...,https://nypost.com/2025/10/24/us-news/ontario-...
271,58,2025-10-24-16-12-41 +0000,latimes,"U.S. sanctions Colombia's president, deploys a...",https://www.latimes.com/world-nation/story/202...
284,58,2025-10-24-15-22-00 +0000,wsj,Annual inflation heated up slightly in Septemb...,https://www.wsj.com/economy/consumer-price-ind...
247,46,2025-10-24-17-43-51 +0000,nypost,Influential House Minority Leader Hakeem Jeffr...,https://nypost.com/2025/10/24/us-news/hakeem-j...
101,43,2025-10-25-03-18-59 +0000,nypost,Department of War receives $130 million from m...,https://nypost.com/2025/10/24/us-news/departme...
18,42,2025-10-25-13-07-30 +0000,bbc,Trump hopes China will help push Russia toward...,https://www.bbc.com/news/articles/cn51yk9vgrko...
77,40,2025-10-25-08-00-00 +0000,cbc,Palestinian man says he wishes he could see fa...,https://www.cbc.ca/news/world/gaza-palestinian...
214,38,2025-10-24-19-35-06 +0000,latimes,Mega Millions alcanza los 680 millones de dóla...,https://www.latimes.com/espanol/california/art...
71,34,2025-10-25-09-00-14 +0000,nypost,"Early voting for NYC mayoral election, other r...",https://nypost.com/2025/10/25/us-news/early-vo...
124,30,2025-10-25-00-55-26 +0000,nypost,Prosecutor who opposed charging Letitia James ...,https://nypost.com/2025/10/24/us-news/prosecut...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
